In [1]:
from sklearn.model_selection import train_test_split

In [2]:
%run Company2CoursesPreprocessing.ipynb

In [3]:
########################### get similarity matrix based on tier and ctc #########################################################
df_company_det1 = df_company_det
df_company_det1 = df_company_det1.reset_index(level=['CompanyName_New'])
df_company_det1['CompanyName_Cat'] = df_company_det1["CompanyName_New"].astype('category').cat.codes
df_company_det1 = df_company_det1.drop(['CompanyName_New'], axis = 1) 
ctcRatingsData = df_company_det1.stack().reset_index().rename(columns={'level_0':'CompanyName_Cat','level_1':'Features', 0:'FeatureRating'})
ctcRatingsData['Features_Cat'] = ctcRatingsData["Features"].astype('category').cat.codes
df_features = ctcRatingsData[['Features_Cat','Features']]
df_features = df_features.drop_duplicates(subset=['Features_Cat','Features'], keep='last')
ctcRatingsData = ctcRatingsData.drop(['Features'], axis = 1) 

In [4]:
dfCourseGradeAvgTemp = dfCourseGradeAvg

In [5]:
dfCourseGradeAvg = dfCourseGradeAvg.reset_index(level=['CompanyName_New'])

In [6]:
dfCourseGradeAvg["CompanyName_Cat"] = dfCourseGradeAvg["CompanyName_New"].astype('category').cat.codes

In [7]:
df_company = dfCourseGradeAvg[['CompanyName_New','CompanyName_Cat']]
df = dfCourseGradeAvg.drop(['CompanyName_New'], axis = 1) 

In [8]:
originalGradeRatingsData1 = df.stack().reset_index().rename(columns={'level_0':'CompanyName_Cat','level_1':'CourseName', 0:'GradeRating'})

In [9]:
originalGradeRatingsData1['CourseName_Cat'] = originalGradeRatingsData1["CourseName"].astype('category').cat.codes

In [10]:
df_grade = originalGradeRatingsData1[['CourseName_Cat','CourseName']]
df_grade = df_grade.drop_duplicates(subset=['CourseName_Cat','CourseName'], keep='last')
originalGradeRatingsData1 = originalGradeRatingsData1.drop(['CourseName'], axis = 1) 

In [11]:
xTrain, xTest = train_test_split(originalGradeRatingsData1, test_size = 0.1, random_state = 0)

In [12]:
train_data_matrix =xTrain.as_matrix(columns = ['CompanyName_Cat', 'CourseName_Cat', 'GradeRating'])
test_data_matrix = xTest.as_matrix(columns = ['CompanyName_Cat', 'CourseName_Cat', 'GradeRating'])

# Check their shape
print(train_data_matrix.shape)
print(test_data_matrix.shape)

(9379, 3)
(1043, 3)


C:\Users\rames\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
C:\Users\rames\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [13]:
from sklearn.metrics.pairwise import pairwise_distances

# User Similarity Matrix
company_correlation = 1 - pairwise_distances(xTrain, metric='correlation')
company_correlation[np.isnan(company_correlation)] = 0
print(company_correlation[:4, :4])

[[1.         0.98067737 0.86070881 0.92134358]
 [0.98067737 1.         0.74448185 0.97959261]
 [0.86070881 0.74448185 1.         0.59509721]
 [0.92134358 0.97959261 0.59509721 1.        ]]


In [14]:
from sklearn.metrics.pairwise import pairwise_distances

# User Similarity Matrix
company_correlation1 = 1 - pairwise_distances(ctcRatingsData, metric='correlation')
company_correlation1[np.isnan(company_correlation1)] = 0
print(company_correlation1[:4, :4])

[[ 1.          0.5        -0.5         0.99615718]
 [ 0.5         1.          0.5         0.42222906]
 [-0.5         0.5         1.         -0.57392812]
 [ 0.99615718  0.42222906 -0.57392812  1.        ]]


In [15]:
def predict(ratings, similarity):
    mean_company_rating = ratings.mean(axis=1)
    # Use np.newaxis so that mean_company_rating has same format as ratings
    ratings_diff = (ratings - mean_company_rating[:, np.newaxis])
    pred = mean_company_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    
    return pred

In [16]:
from sklearn.metrics import mean_squared_error
from math import sqrt

# Function to calculate RMSE
def rmse(pred, actual):
    # Ignore nonzero terms.
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return sqrt(mean_squared_error(pred, actual))

In [17]:
########### company similarity just based on rating ##############################################################################

company_based_prediction = predict(train_data_matrix, company_correlation)
# RMSE on the test data
print('test Company-based CF RMSE: ' + str(rmse(company_based_prediction, test_data_matrix)))
# RMSE on the train data
print('train Company-based CF RMSE: ' + str(rmse(company_based_prediction, train_data_matrix)))


test Company-based CF RMSE: 47.66636367211358
train Company-based CF RMSE: 24.361409486898452


In [22]:
####################### using model based approach #######################################

# mapping the matrix into a 3 column dataframe=> using originalRatingsData from Company2CoursesPreprocessing.ipynb
# using result (sparse matrix from Company2CoursesPreprocessing.ipynb )
dfCourseGradeAvg = dfCourseGradeAvgTemp.fillna(0)
R = dfCourseGradeAvg.as_matrix()
company_ratings_mean = np.mean(R, axis = 1)
Ratings_demeaned = R - company_ratings_mean.reshape(-1, 1)

# applying SVD ( will be useful when datapoints increase)
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(Ratings_demeaned, k = 50)
sigma = np.diag(sigma)
all_company_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + company_ratings_mean.reshape(-1, 1)

# getting the statistical predicted ratings, will be helpful in case certain courses were not mentioned
preds = pd.DataFrame(all_company_predicted_ratings, columns = dfCourseGradeAvg.columns)
preds.head()

C:\Users\rames\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


,compiler_design,computer_networks,engineering_mathematics_i,basic_electrical_engineering,research_credits,linear_algebra_and_its_applications,machine_learning,project_work,engineering_mathematics_ii,internship,...,mechanics_of_solids,aerospace_structures,basics_of_automobile_engineering,mechanics_of_machines_and_mechanisms,heat_transfer,introduction_to_gas_dynamics,hands_on_course_on_digital_system_simulation_and_synthesis,linear_system,embedded_programming_with_arduino,adaptive_system
0,3.514651,4.581427,3.805836,3.617477,-0.003592,3.801515,2.750724,4.267094,3.612864,5.252830,...,-0.001239,-0.001239,-0.001239,-0.001354,-0.001296,-0.001296,0.002870,0.024001,-0.006265,0.005440
1,4.053069,5.059424,5.062149,5.073230,5.093883,5.998297,3.956107,5.058295,4.000892,6.006821,...,0.002643,0.002643,0.002643,-0.005775,-0.001566,-0.001566,0.004559,0.106599,-0.039459,-0.000945
2,4.910529,5.536663,4.755013,4.644654,6.070466,5.086294,5.332891,5.999276,4.607347,5.998007,...,0.004602,0.004602,0.004602,-0.000710,0.001946,0.001946,-0.003585,0.015380,-0.011722,-0.013148
3,5.339544,4.851410,4.717453,4.579005,0.000283,5.143378,5.199521,0.007185,5.004681,0.001142,...,-0.000405,-0.000405,-0.000405,-0.000589,-0.000497,-0.000497,0.001118,0.010506,5.997061,0.001957
4,4.425307,5.669236,5.994372,4.916740,0.081207,5.488047,3.968944,-0.089283,5.407528,-0.016442,...,0.010511,0.010511,0.010511,0.006388,0.008449,0.008449,-0.018064,-0.110797,0.022077,-0.039887


In [28]:
# Import libraries from Surprise package
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

# Load Reader library
reader = Reader()

# Load ratings dataset with Dataset library
data = Dataset.load_from_df(originalGradeRatingsData1[['CompanyName_Cat', 'CourseName_Cat', 'GradeRating']], reader)

# Split the dataset for 5-fold evaluation
algo = SVD()

# Run 5-fold cross-validation and then print results
cross_validate(algo, data, measures=['RMSE'], cv=5)


{'test_rmse': array([2.30504568, 2.28192832, 2.52163674, 2.11127601, 2.69491385]),
 'fit_time': (1.2403035163879395,
  1.233839511871338,
  1.1996829509735107,
  1.0967950820922852,
  1.14650559425354),
 'test_time': (0.03340721130371094,
  0.033736228942871094,
  0.027959346771240234,
  0.03312397003173828,
  0.027968406677246094)}

In [30]:
trainset = data.build_full_trainset()
algo.fit(trainset)
algo.predict('samsung', 'machine_learning')